<a href="https://colab.research.google.com/github/anky19698/Cricket_Analytics/blob/main/Fantasy_Team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pulp

In [4]:
import pulp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# to ignore warnings
import warnings
warnings.filterwarnings('ignore')

# to display all rows columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

# Fantasy Team Selection

In [40]:
def select_fantasy_team(df):
  df = df.join(pd.get_dummies(df[['Role', 'Team']]))
  df['PlayingXI'] = 1

  teams = list(df.Team.unique())
  team1, team2 = teams[0], teams[1]

  # define decision variable
  pickup_status = pulp.LpVariable.dicts('pickup_status',((obj) for obj in df.index), cat = 'Binary' )

  # define objective function
  mo = pulp.LpProblem('Profit maximisation problem', pulp.LpMaximize)
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj),'Sel'] for obj in df.index])

  # define constraints
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Credits']] for obj in df.index) <= 100

  # 11 players
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'PlayingXI']] for obj in df.index) == 11

  # team constraints
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Team_{team1}"]] for obj in df.index) >= 4
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Team_{team1}"]] for obj in df.index) <= 7

  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Team_{team2}"]] for obj in df.index) >= 4
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), f"Team_{team2}"]] for obj in df.index) <= 7

  # role constraints
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) >= 1
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_WK']] for obj in df.index) <= 4

  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) >= 3
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BAT']] for obj in df.index) <= 6

  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) >= 3
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_BOWL']] for obj in df.index) <= 6

  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) >= 1
  mo += pulp.lpSum([pickup_status[obj]*df.loc[(obj), 'Role_AR']] for obj in df.index) <= 4

  mo.solve()

  # output as a column in df
  df['pickup_status'] = 0

  for obj in df.index:
      if pickup_status[obj].varValue:
          df['pickup_status'][obj] = 1

  fdf = df[df['pickup_status'] == 1].sort_values(['Role_BAT', 'Role_WK', 'Role_AR', 'Role_BOWL'], ascending = False).reset_index(drop = True)
  return fdf

# IND vs SL 1st T20I 24th February 2022

In [13]:
df1 = pd.read_csv('ind_sl_t20i_1st_2022_02_24.csv')

df1.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,PlayingXI,Points,DT
0,3692,38632,I Kishan,10276,IND,2,WK,73.23,8.5,0,1,141,1
1,3692,38632,S Iyer,9428,IND,2,BAT,71.23,8.5,0,1,84,1
2,3692,38632,B Kumar,1726,IND,2,BOWL,22.17,8.5,0,1,76,1
3,3692,38632,C Asalanka,10934,SL,5,BAT,64.60,8.5,0,1,70,1
4,3692,38632,R Sharma,576,IND,2,BAT,87.30,10.5,0,1,66,1


In [41]:
ft1 = select_fantasy_team(df1)

In [42]:
sum(ft1.Credits)

99.5

In [43]:
ft1[['Player', 'PlayerId', 'Team', 'Role', 'Sel', 'Credits', 'Series Points', 'PlayingXI', 'pickup_status', 'Points']]

,Player,PlayerId,Team,Role,Sel,Credits,Series Points,PlayingXI,pickup_status,Points
0,S Iyer,9428,IND,BAT,71.23,8.5,0,1,1,84
1,C Asalanka,10934,SL,BAT,64.60,8.5,0,1,1,70
2,R Sharma,576,IND,BAT,87.30,10.5,0,1,1,66
3,P Nissanka,13682,SL,BAT,85.62,9.5,0,1,1,2
4,I Kishan,10276,IND,WK,73.23,8.5,0,1,1,141
5,K Mendis,9494,SL,WK,53.58,9.0,0,1,1,0
6,R Jadeja,587,IND,AR,74.73,9.0,0,1,1,34
7,W Hasaranga,10926,SL,AR,69.76,9.5,0,1,1,0
8,D Chameera,8393,SL,BOWL,78.22,9.0,0,1,1,30
9,H Patel,8175,IND,BOWL,65.20,8.5,0,1,1,8


In [19]:
m1 = sum(ft1.Points) + ft1.Points[0] + 0.5*ft1.Points[1]
m1

560.0

# IND vs SL 2nd T20I 26th February 2022

In [44]:
df2 = pd.read_csv('ind_sl_t20i_2nd_2022_02_26.csv')

df2.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points,DT
0,3692,38637,S Iyer,9428,IND,2,BAT,87.64,8.5,84,1,1,112,1
1,3692,38637,P Nissanka,13682,SL,5,BAT,66.77,9.5,2,0,1,100,1
2,3692,38637,R Jadeja,587,IND,2,AR,76.24,9.0,34,1,1,93,1
3,3692,38637,D Shanaka,8422,SL,5,AR,68.26,9.0,32,1,1,77,1
4,3692,38637,S Samson,8271,IND,2,WK,25.94,8.0,0,0,1,59,1


In [46]:
ft2 = select_fantasy_team(df2)

In [47]:
sum(ft2.Credits)


99.5

In [48]:
m2 = sum(ft2.Points) + ft2.Points[0] + 0.5*ft2.Points[1]

m2

684.0

# IND vs SL 3rd T20I 27th February 2022

In [49]:
df3 = pd.read_csv('ind_sl_t20i_3rd_2022_02_27.csv')

df3.head()

,SeriesId,MatchId,Player,PlayerId,Team,TeamId,Role,Sel,Credits,Series Points,Series DT,PlayingXI,Points
0,3692,38642,S Iyer,9428,IND,2,BAT,88.8,8.5,196.0,2,1,100
1,3692,38642,R Jadeja,587,IND,2,AR,87.6,9.0,127.0,2,1,31
2,3692,38642,S Samson,8271,IND,2,WK,85.5,8.0,59.0,1,1,35
3,3692,38642,D Shanaka,8422,SL,5,AR,82.3,9.0,109.0,2,1,105
4,3692,38642,P Nissanka,13682,SL,5,BAT,82.0,9.5,102.0,1,1,-1


In [50]:
ft3 = select_fantasy_team(df3)

In [51]:
m3 = sum(ft3.Points) + ft3.Points[0] + 0.5*ft3.Points[1]

m3

510.5

# Average of 3 Matches

In [52]:
(m1+m2+m3)/3

584.8333333333334